# Automated Inventory Management with RPA in Python 

This project utilizes robotic process automation (RPA) and Python to automate inventory management tasks. 

The program is capable of adding new items to the inventory, updating existing items, and deleting items from the inventory based on information extracted from PDF files. 

The PDF files are parsed using the PyPDF2 module and regular expressions to extract product name, seller, and price. The rpa module is used for the RPA tasks, such as opening a URL, clicking buttons, typing into input fields, and taking a screenshot of the updated inventory table. 

By automating these tasks, this project can greatly reduce the time and effort required for inventory management.

In [ ]:
import rpa as r    # import the rpa module for robotic process automation
import PyPDF2     # import PyPDF2 for PDF manipulation
import re    # import regular expression module for text parsing
import glob   # import glob module for file handling

# function to extract data from a PDF file
def data_extract(pdfpath):
    # open the Run dialog and input the PDF path
    r.keyboard('[win]r')
    r.clipboard(pdfpath)
    r.keyboard('[clear][ctrl]v [enter]')
    r.wait(5)

    # select all text in the opened PDF and copy to clipboard
    r.keyboard('[ctrl]a')
    r.keyboard('[ctrl]c')
    r.keyboard('[ctrl]q')
    data=r.clipboard()

    # extract the product name, seller, and price from the copied text using regex
    product_name=re.findall('(?<=Product Name : ).*',data)[0].strip()
    seller=re.findall('(?<=Seller : ).*',data)[0].replace("\n","").strip()
    price=re.findall('(?<=Price : ).*',data)[0].strip()

    # return the extracted data as a list
    return [product_name,seller,price]

# initialize rpa with visual automation and Chrome browser
r.init(visual_automation = True, chrome_browser = True)

# open the URL for the inventory management system
r.url('https://excelcult.com/inventorymanagement/')

# click the delete all button to clear the table
r.click('//button[@id="btn-delete"]')

# loop through all PDF files in the insert directory and add new items to the inventory
insert_files=glob.glob('insert\\*.pdf')
for f in insert_files:
    # extract data from the PDF file
    data= data_extract(f)
    # input the data into the corresponding input fields in the web form
    r.type('//input[@id="proname"]',data[0])
    r.type('//input[@id="seller"]',data[1])
    r.type('//input[@id="price"]',data[2])
    # click the create button to add the new item to the inventory
    r.click('//button[@id="btn-create"]')

# loop through all PDF files in the update directory and update existing items in the inventory
update_files=glob.glob('update\\*.pdf')
for f in update_files:
    # extract data from the PDF file
    data= data_extract(f)
    # find the corresponding row in the table based on the product name and seller
    xpath='//tr[td[2][text()="'+data[0]+'"] and td[3][text()="'+data[1]+'"]]/td[5]/img'
    # click the edit button for the row and input the updated price
    r.click(xpath)
    r.type('//input[@id="price"]','[clear]'+data[2])
    # click the update button to save the changes
    r.click('//button[@id="btn-update"]')

# loop through all PDF files in the delete directory and delete corresponding items from the inventory
delete_files=glob.glob('delete\\*.pdf')
for f in delete_files:
    # extract data from the PDF file
    data= data_extract(f)
    # find the corresponding row in the table based on the product name and seller
    xpath='//tr[td[2][text()="'+data[0]+'"] and td[3][text()="'+data[1]+'"]]/td[6]/img'
    # Click the delete button for the row to remove the item from the inventory
    r.click(xpath)

# Click the read button to refresh the table
r.click('//button[@id="btn-read"]')

# Take a screenshot of the updated table
r.snap('//table[@class="table table-striped"]','results.png')

# Close the browser
r.close()

# Open the screenshot of the table
r.run('start results.png')